In [1]:
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from keras.callbacks import TensorBoard
from keras import optimizers

from tensorflow import keras
from keras.models import Sequential
from keras.layers import *
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

import pickle
import time

#tf.compat.v1.disable_eager_execution()

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=90,
        validation_split=0.15)

#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-detection/training data/images2",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-detection/training data/images2",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

'''test_generator = test_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-classification/images/test data",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42
)'''

NAME = "defect-classification-CNN-64*2-{}".format(int(time.time()))


model = Sequential()

model.add(VGG16(include_top=False, weights='imagenet', input_shape = (224,224, 3)))

'''model.add(Conv2D(64, (3, 3), padding='same', input_shape = (224, 224, 3)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))'''

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

'''model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))'''

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))
#model.add(BatchNormalization())

model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary ()

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'],
              )


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = checkpoint

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=20,
                    callbacks=[callbacks_list, tensorboard])
                    


/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

Found 17850 images belonging to 2 classes.
Found 3150 images belonging to 2 classes.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         589952    
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 256)         295168    
__________________________________

In [2]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

[0.00042614605627022684, 0.9996811151504517]

In [3]:
from keras.models import load_model 
model.save("defect-detection_model.h5")
model.save_weights("defect-detection_weights.h5")

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-detection/test data/images",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42
)

Found 222 images belonging to 2 classes.


In [5]:
test_datagen2 = ImageDataGenerator(rescale=1./255)

test_generator2 = test_datagen2.flow_from_directory(
    directory=r"/home/trojan/Desktop/defect-detection/test data_2",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42
)

Found 222 images belonging to 1 classes.


In [6]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [7]:
model.evaluate_generator(generator=test_generator,
steps=STEP_SIZE_TEST)

[0.0011406027479097247, 1.0]

In [8]:
STEP_SIZE_TEST2=test_generator2.n//test_generator2.batch_size

In [9]:

test_generator2.reset()
pred=model.predict_generator(test_generator2,
steps=STEP_SIZE_TEST,
verbose=1)

222/222 [==============================] - 4s 19ms/step


In [10]:
import numpy as np
import pandas as pd

predicted_class_indices=np.argmax(pred,axis=1)

cl = np.round(pred)


In [12]:
idx_to_cls = {v: k for k, v in valid_generator.class_indices.items()}
preds_cls = np.vectorize(idx_to_cls.get)(cl[:,0])
filenames_to_cls = list(zip(test_generator2.filenames))

results=pd.DataFrame({ "file":filenames_to_cls,"pr":pred[:,0], "class": cl[:,0], "class_name": preds_cls})

results.to_csv("results.csv",index=False)

In [32]:
from PIL import Image, ImageDraw

width = 350
height = 400

im = Image.new('RGB', (width, height), (128, 128, 128))
draw = ImageDraw.Draw(im)

draw.rectangle((0, height, width, 0), fill=(255, 255, 255), outline=(0, 0, 0)) # white, black
im.show()

In [ ]:
import numpy as np
import cv2

# Create a black image
img = np.zeros((512,512,3), np.uint8)
img = cv2.rectangle(img,(384,0),(510,128),(0,255,0),3)
cv2.imshow('image',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import re
re.split("[(,..)]", "point(1509,1301).csv.jpg")[-4]